In [5]:
import pandas as pd
import os
from datetime import datetime
import cx_Oracle
from openpyxl import load_workbook

def FETCH_DATA():
    # Define connection details
    dsn = cx_Oracle.makedsn(
        host = "192.168.1.242",      # Replace with your host IP or domain
        port = 1526,                # Replace with your port
        service_name = "sperpdb"  # Replace with your service name
    )

    # Establish the connection
    connection = cx_Oracle.connect(
        user = "spselect",         # Replace with your username
        password = "select",     # Replace with your password
        dsn = dsn
    )
    query = """
        SELECT * FROM ssl_cst_orde_d 
        WHERE SC_NO IN ('25070022', '25070023', '25070024', '25070025')
    """
    df = pd.read_sql_query(query, connection)
    connection.close()
    return df
    

Order_List = FETCH_DATA()
select_columns = ["SC_NO", "PDC_3", "ORDER_QTY"]
Order_List = Order_List[select_columns].copy()

C:\Users\sales6\AppData\Local\Temp\ipykernel_14416\1755540040.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [6]:
base_path = r"Z:\業務部\業務一課\H-訂單\1. 外銷\D09200 National Nail\1. 訂單\2025"

CNT_amount = Order_List.drop_duplicates(subset=["SC_NO"])
today_str = datetime.today().strftime("%Y%m%d")
folder_name = f"{today_str} 庫存單{CNT_amount.shape[0]}櫃"

create_folder = os.path.join(base_path, folder_name)
os.makedirs(create_folder, exist_ok=True)

In [7]:
Order_List = Order_List.rename(columns={'PDC_3': '尺寸', 'ORDER_QTY': '訂單項次M數'})

Order_List['尺寸'] = Order_List['尺寸'].replace({
    '01200': '9x1-1/4',
    '01500': '9x1-5/8',
    '02200': '9x2-1/4'
})

file_path = os.path.join(create_folder, "stock_order.xlsx")
Order_List.to_excel(file_path, index=False)

grouped = Order_List.groupby("尺寸")["訂單項次M數"].sum()
grouped

尺寸
9x1-1/4    13894.4
9x1-5/8     7443.2
9x2-1/4      192.0
Name: 訂單項次M數, dtype: float64

In [8]:
wb = load_workbook(file_path)
ws = wb['Sheet1']

ws.insert_rows(1)
# Add the title
ws['A1'] = "庫存單明細"

# Write grouped summary below Order_List
start_row = Order_List.shape[0] + 4
ws[f'B{start_row}'] = '尺寸'
ws[f'C{start_row}'] = '總M數'

for i, (size, total) in enumerate(grouped.items(), start=start_row + 1):
    ws[f'B{i}'] = size
    ws[f'C{i}'] = total

# Save changes
wb.save(file_path)